# Usuarios y títulos

In [1]:
import ast
import csv
import itertools
import os
import re
from collections import Counter
from collections import defaultdict
from datetime import datetime
from datetime import timedelta
from string import punctuation

import matplotlib
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
from pandas.tslib import Timestamp

## Préstamos

In [2]:
%%time

merge_df = pd.read_csv('merge.csv', header=0)
merge_df = merge_df.fillna('')

CPU times: user 1.62 s, sys: 45.1 ms, total: 1.67 s
Wall time: 1.67 s


In [3]:
merge_df.shape

(516206, 12)

In [4]:
merge_df.head()

,TITULO,AUTOR,DEWEY,FECHA_PUB,PAIS_EJEMPLAR,COD_BARRAS_LECTOR,CODBAR,Sucursal,Tratamiento,Edad,Ciudad,Rango_edad
0,Johannes Gutenberg,"Borràs Perelló, Lluís.",925,2010,Spain,88007424,88007424,ARM,sr.,43,Armenia,40-49
1,Genealogía de una bruja,"Lacombe, Benjamin, 1982-",843,2009,Spain,88007424,88007424,ARM,sr.,43,Armenia,40-49
2,La independencia de Colombia : así fue,"Vasco, Irene, 1952-",986,2009,Colombia,88007424,88007424,ARM,sr.,43,Armenia,40-49
3,Maravillas del mundo,"Steele, Philip.",910,2007,Mexico,88007424,88007424,ARM,sr.,43,Armenia,40-49
4,No te gustaría ser un adivino maya,"Matthews, Rupert.",972,2008,Colombia,88007424,88007424,ARM,sr.,43,Armenia,40-49


## Thresholds

In [5]:
THRESHOLD_GROUPS = 100

In [6]:
THRESHOLD_TITLES = 5

In [7]:
TOP_GROUP = 10

## Groups

In [8]:
group_count_sr = merge_df.groupby(['Ciudad', 'Tratamiento', 'Rango_edad']).CODBAR.count()

In [9]:
len(group_count_sr)

333

In [10]:
sorted_group_count_sr = group_count_sr.sort_values(ascending=False)

In [11]:
pd.DataFrame(sorted_group_count_sr)

CODBAR
Ciudad       Tratamiento Rango_edad        
Bogotá       sr.         18-29        46986
             sra.        18-29        42716
             sr.         30-39        31544
Tunja        sra.        -17          26232
Bogotá       sra.        30-39        22022
Tunja        sr.         -17          17622
Bogotá       sr.         50-64        16693
                         40-49        16089
Popayán      sra.        -17          12217
             sr.         -17           8630
Pasto        sra.        -17           7970
             sr.         -17           7738
Bogotá       sra.        40-49         7399
Tunja        sra.        18-29         6745
Bogotá       sra.        50-64         6483
Neiva        sr.         18-29         5628
Tunja        sr.         18-29         5503
Pereira      sra.        -17           5347
Pasto        sr.         18-29         5248
Popayán      sr.         18-29         5092
Pasto        sra.        18-29         5081
Bogotá       sr.         +65           5003
Tunja        sra.        30-39         4809
Popayán      sra.        18-29         4729
Ipiales      sra.        -17           4277
             sr.         -17           3834
Neiva        sra.        18-29         3405
Pereira      sr.         18-29         3352
                         -17           3247
Pasto        sra.        30-39         3188
...                                     ...
Quibdó       sr.         40-49           20
Manizales    sra.        +65             16
Valledupar   sra.        50-64           16
Quibdó       sra.        40-49           14
Montería     sr.         -17             13
Ibagué       sra.        +65             13
Cúcuta       sr.         18-29           12
Cali         sra.        40-49           12
Cúcuta       sra.        40-49           11
Buenaventura sra.        50-64            9
Barranquilla sra.        +65              9
Manizales    sra.        -17              9
Medellín     sra.        -17              8
Popayán      sra.        +65              8
Cali         sra.        50-64            7
Cúcuta       sra.        +65              6
                         30-39            6
Valledupar   sra.        +65              6
Cúcuta       sra.        50-64            5
Medellín     sr.         -17              4
Florencia    sra.        +65              4
San Andrés   sra.        50-64            3
Leticia      sra.        40-49            3
Montería     sra.        -17              3
Quibdó       sra.        50-64            3
Girardot     sra.        +65              3
Buenaventura sra.        +65              3
Leticia      sra.        +65              2
Sincelejo    sra.        +65              2
Montería     sra.        50-64            1

[333 rows x 1 columns]

In [12]:
filtered_group_count_sr = group_count_sr[lambda x: x >= THRESHOLD_GROUPS]

In [13]:
len(filtered_group_count_sr)

245

In [14]:
sorted_filtered_group_count_sr = filtered_group_count_sr.sort_values(ascending=False)

In [15]:
pd.DataFrame(sorted_filtered_group_count_sr.head())

CODBAR
Ciudad Tratamiento Rango_edad        
Bogotá sr.         18-29        46986
       sra.        18-29        42716
       sr.         30-39        31544
Tunja  sra.        -17          26232
Bogotá sra.        30-39        22022

## Titles

In [16]:
title_count_sr = merge_df.groupby(['Ciudad', 'Tratamiento', 'Rango_edad', 'TITULO']).CODBAR.count()

In [17]:
len(title_count_sr)

321402

In [18]:
sorted_title_count_sr = title_count_sr.sort_values(ascending=False)

In [19]:
pd.DataFrame(sorted_title_count_sr)

CODBAR
Ciudad    Tratamiento Rango_edad TITULO                                                    
Bogotá    sr.         18-29      Obras completas                                        118
          sra.        18-29      Obras completas                                         72
          sr.         30-39      Obras completas                                         67
Neiva     sr.         18-29      Principios de anatomía y fisiología                     60
Tunja     sra.        -17        Orejitas                                                54
Cartagena sr.         50-64      Física general : con experimentos sencillos             53
Pereira   sr.         18-29      En el trayecto ... aprende escuchando dónde qui...      50
Neiva     sra.        -17        Un cuento bilingüe para cada día = A bilingual ...      49
Tunja     sra.        -17        La escoba de la viuda                                   49
                                 Gorila                                                  48
Bogotá    sr.         18-29      Física                                                  48
                                 Cien años de soledad                                    47
Neiva     sra.        18-29      Principios de anatomía y fisiología                     47
Tunja     sra.        -17        Había una vez una [princesa]                            46
Bogotá    sra.        40-49      La política pública de desplazamiento forzado e...      45
Tunja     sra.        -17        Había una vez una [llave]                               45
                                 El libro de los cerdos                                  44
                                 Agu trot                                                43
                                 Cuentos pintados                                        42
Pereira   sra.        -17        Un cuento bilingüe para cada día = A bilingual ...      42
Bogotá    sra.        18-29      Cien años de soledad                                    41
Neiva     sr.         30-39      Geometría básica                                        40
Tunja     sra.        -17        La campeona mundial de mantenerse despierta             40
Florencia sr.         50-64      Método y conocimiento : metodología de la inves...      40
Tunja     sra.        -17        La peor señora del mundo                                40
Bogotá    sr.         18-29      Teatro completo                                         40
Tunja     sr.         -17        Willy el mago                                           39
Bogotá    sr.         18-29      Manual de derecho penal : parte general                 39
          sra.        40-49      Estado del arte de las metodologías de trabajo ...      39
          sr.         18-29      Diálogos                                                38
...                                                                                     ...
Ipiales   sra.        -17        La conspiración de las tías                              1
                                 La embajada de Rui                                       1
                                 La emancipada : primera novela ecuatoriana               1
                                 La educación infantil : métodos, técnicas y org...       1
                                 La doncella de agua : (tragedia en dos actos)            1
                                 La dificil claridad                                      1
                                 La dictadura de Bolívar                                  1
                                 La desnudez antigua                                      1
                                 La depresión : cómo superarla                            1
                                 La dama boba                                             1
                                 La cómica historia del mercader de Venecia, tam...       1
                                 La c

In [20]:
filtered_title_count_sr = title_count_sr[lambda x: x >= THRESHOLD_TITLES]

In [21]:
len(filtered_title_count_sr)

12853

In [22]:
sorted_filtered_title_count_sr = filtered_title_count_sr.sort_values(ascending=False)

In [23]:
pd.DataFrame(sorted_filtered_title_count_sr.head())

CODBAR
Ciudad Tratamiento Rango_edad TITULO                                     
Bogotá sr.         18-29      Obras completas                         118
       sra.        18-29      Obras completas                          72
       sr.         30-39      Obras completas                          67
Neiva  sr.         18-29      Principios de anatomía y fisiología      60
Tunja  sra.        -17        Orejitas                                 54

## Count

### Global

In [24]:
%%time

ciudades = merge_df.Ciudad.unique()
tratamientos = merge_df.Tratamiento.unique()
rangos_edades = merge_df.Rango_edad.unique()

indexes = []
for ciudad in ciudades:
    for tratamiento in tratamientos:
        for rango_edad in rangos_edades:
            index = (ciudad, tratamiento, rango_edad)
            if index in filtered_group_count_sr and index in filtered_title_count_sr:
                sr = title_count_sr[index]
                sorted_sr = sr.sort_values(ascending=False).head(TOP_GROUP)
                for title in sorted_sr.index:
                    indexes.append(index + (title,))

CPU times: user 1.17 s, sys: 7.11 ms, total: 1.17 s
Wall time: 1.17 s


In [25]:
selected_title_count_sr = title_count_sr[indexes]

In [26]:
len(selected_title_count_sr)

2100

In [27]:
selected_title_count_sr.head()

Ciudad   Tratamiento  Rango_edad  TITULO                       
Armenia  sr.          -17         Calvin y hobbes                  15
                                  Vuelo 714 para Sidney            13
                                  Tiranosaurio : lagarto tirano    11
                                  Los cigarros del faraón          11
                                  ¿Dónde está Wally?               11
Name: CODBAR, dtype: int64

### Local

In [28]:
from IPython.display import display

In [29]:
pd.set_option('display.max_rows', 120)

def process_group_count(x):
    index = x.name[:2]
    count = group_count_sr[(ciudad, ) + index]
    return count

ciudades = selected_title_count_sr.index.levels[0]

for ciudad in ciudades:
    sr = selected_title_count_sr[ciudad]
    df = pd.DataFrame(sr)
    df['Cuenta_grupo'] = df.apply(process_group_count, axis=1)
    df = df.rename(columns={'CODBAR': 'Cuenta_titulo'})
    print(ciudad)
    display(df)

pd.reset_option('display.max_rows')

Armenia


Cuenta_titulo  \
Tratamiento Rango_edad TITULO                                                              
sr.         -17        Calvin y hobbes                                                15   
                       Vuelo 714 para Sidney                                          13   
                       Tiranosaurio : lagarto tirano                                  11   
                       Los cigarros del faraón                                        11   
                       ¿Dónde está Wally?                                             11   
                       El templo del sol                                              10   
                       Encuentra los animales.                                        10   
                       Gandhi                                                         10   
                       Un cuento bilingüe para cada día = A bilingual ...             10   
                       Tintin en América                                               9   
            30-39      La magia de la plastilina                                       7   
                       Pensar sistémico : una introducción al pensamie...              5   
                       Mi PC : actualización, configuración, mantenimi...              4   
                       La gestión de la tecnología : elementos fundame...              3   
                       El inventor de mamás                                            3   
                       El río : exploraciones y descubrimientos en la ...              3   
                       The avengers                                                    3   
                       Los increibles                                                  3   
                       Las mil y una noches                                            3   
                       Actualización, mantenimiento y solución de prob...              3   
            +65        Noticias historiales de las conquistas de tierr...              6   
                       Quindío 1840-1950 : historia de la diferencia s...              5   
                       La mente nueva del emperador : en torno a la ci...              4   
                       Armenia : 1935-1936                                             4   
                       Geografía e historia de la provincia del Quindí...              4   
                       La industria mundial del café                                   4   
                       Diálogos cafeteros                                              4   
                       Fenalco y el Quindío                                            3   
                       Mr Mercedes                                                     3   
                       Obras completas                                                 3   
            18-29      Animales y derecho = Animals and the law.                       6   
                       Historia de la Arquitectura                                     4   
                       Los estándares de la Corte Interamericana y la ...              3   
                       Cartas a un joven poeta                                         3   
                       Mes amours                                                      2   
                       La teoría de las formas de gobierno en la histo...              2   
                       El oriente es un mito                                           2   
                       El olvido que seremos                                           2   
                       Régimen municipal y departamental : anotado                     2   
                       Cien años de soledad                                            2   
sra.        40-49      La tregua                                                       5   
                       El hombre araña spider-man                                      4   
                       Cuen

Barranquilla


Cuenta_titulo  \
Tratamiento Rango_edad TITULO                                                              
sr.         40-49      El capo 2                                                      20   
                       El capo                                                        18   
                       El guardián de la salud natural y alternativa                  17   
                       13 actuaciones de los 3 chiflados                               6   
                       Forrest Gump                                                    6   
                       Administración estratégica de marca : branding                  5   
                       Ratatouille                                                     5   
                       Dinosaurio                                                      5   
                       Razonamiento abstracto                                          5   
                       Los mitos griegos                                               5   
            30-39      Manual de derecho penal : parte general                         5   
                       Reminiscencias de Santafé y Bogotá                              4   
                       Historia de la pedagogía                                        4   
                       Obra completa                                                   3   
                       Historia del cuerpo                                             3   
                       Estética : la cuestión del arte                                 3   
                       Estigma : la identidad deteriorada                              3   
                       Historia de la filosofía del lenguaje                           3   
                       Filosofía sin supuestos : de Husserl a Heidegger                3   
                       La pedagogía contemporánea                                      3   
            50-64      No hay silencio que no termine : Ingrid Betancourt             19   
                       Las guerras de la paz                                          13   
                       El costeñol : un dialecto con toda la barba                    11   
                       También la oscuridad tiene su sombra : cuentos                  7   
                       Historia de la filosofía                                        5   
                       Mi cumpleaños                                                   4   
                       El amor en los tiempos del cólera                               4   
                       El clan de los doce apóstoles : conversaciones ...              4   
                       Historia musical de Rufo Garrido                                4   
                       Bovea y Sus Vallenatos 40 éxitos originales.                    4   
            18-29      Termodinámica                                                  16   
                       Diseño en ingeniería mecánica de Shigley                       13   
                       Fenomenos de transporte                                        12   
                       El amor en los tiempos del cólera                              11   
                       Diseño de elementos de máquinas                                 8   
                       Probabilidad y estadística aplicadas a la ingen...              6   
                       Vivir para contarla                                             5   
                       Barrio abajo : el barrio de donde somos todos                   5   
                       Mecánica de materiales                                          5   
                       Fisicoquímica                                                   5   
sra.        18-29      Los caminos de la libertad                                      6   
                       Cárcel y derechos humanos : un enfoque relativo...              5   
                       La t

Bogotá


...

Cuenta_titulo  \
Tratamiento Rango_edad TITULO                                                              
sr.         40-49      La guía completa sobre instalaciones eléctricas                18   
                       Instalaciones eléctricas de baja tensión                       12   
                       Los siglos decisivos : la experiencia medieval                  9   
                       El libro egipcio de los muertos : el Papiro de Ani              9   
                       Diseño computarizado de carreteras                              6   
                       La frontera de los Llanos en la historia de Col...              6   
                       Derecho administrativo : general y colombiano                   6   
                       Introducción a la psicología                                    5   
                       Martín Fierro                                                   5   
                       Teoría histórico-genética de la cultura : la ló...              5   
            -17        Willy el mago                                                  39   
                       El templo del sol                                              34   
                       Donde viven los monstruos                                      33   
                       Había una vez una [llave]                                      31   
                       Alex quiere un dinosaurio                                      30   
                       Las joyas de la castafiore                                     29   
                       El monstruo que se comió la oscuridad                          29   
                       Están solos en casa dos ositos                                 28   
                       El libro de los cerdos                                         28   
                       Willy el tímido                                                27   
            30-39      Código de procedimiento administrativo y de lo ...             14   
                       Código laboral : sustantivo del trabajo y proce...             12   
                       Diario                                                         11   
                       Mecánica de materiales                                          8   
                       Don Quijote de la Mancha 39 episodios                           8   
                       Escalona un canto a la vida                                     8   
                       Poesías y otros textos                                          7   
                       Biblioteca de comunicación                                      7   
                       Orden y violencia : evolución socio-política de...              7   
                       Derecho de sucesiones                                           7   
            50-64      Un lobo así de grande                                          22   
                       ¡Todo el mundo va!                                             19   
                       Lobo                                                           17   
                       La tortuga vs. la liebre                                       16   
                       La peor señora del mundo                                       15   
                       Debajo de la cama                                              15   
                       Cuentos para niños : que se duermen enseguida                  13   
                       Corso multimediale di lingua e civiltà italiana...             12   
                       La mosca                                                       11   
                       El retal                                                       10   
            +65        Chemical looping systems for fossil energy conv...              5   
                       Los salmos del rey David primera parte salmos 1...              3   
                       Prov

Valledupar


Cuenta_titulo  \
Tratamiento Rango_edad TITULO                                                              
sr.         40-49      Introducción a la ingeniería y al diseño en la ...              5   
                       Introducción al derecho                                         4   
                       El derecho de los jueces : obligatoriedad del p...              3   
                       Macroeconomía                                                   3   
                       Teoría de control para informáticos                             3   
                       Código general del proceso y de procedimiento c...              3   
                       Artefactos : diseño conceptual                                  3   
                       Fundamentos de dibujo : completo curso profesio...              3   
                       Compendio de derecho procesal : pruebas judicia...              2   
                       El mundo de Sofía : novela sobre la historia de...              2   
            -17        Pequeños creativos                                              5   
                       Hermano lobo : crónicas de la prehistoria                       5   
                       Para comprender el universo                                     4   
                       Insurgente                                                      4   
                       Harry Potter y la cámara secreta                                4   
                       La teoría del todo : el origen y el destino del...              4   
                       Satanás                                                         4   
                       Los cigarros del faraón                                         4   
                       En el trayecto ... aprende escuchando dónde qui...              3   
                       Harry Potter y la piedra filosofal                              3   
            30-39      Instituciones jurídico-procesales del trabajo y...              9   
                       Guía teórica y práctica de derecho procesal del...              7   
                       Summa de Maqroll el Gaviero : poesía reunida                    5   
                       Estudios de derecho procesal administrativo : L...              5   
                       Manual de fertilizantes para cultivos de alto r...              4   
                       Práctica administrativa : comentarios y análisi...              4   
                       Sentencias : herejías constitucionales                          3   
                       Instalaciones eléctricas de baja tensión                        3   
                       Derecho administrativo general y colombiano : c...              3   
                       Ibis                                                            3   
            50-64      Código de Procedimiento Administrativo y de lo ...              7   
                       Derecho de sucesiones                                           4   
                       Introducción a las finanzas                                     3   
                       Breve historia de la medicina                                   3   
                       Código de procedimiento administrativo y de lo ...              3   
                       Derecho de sucesiones : teórico práctico aprend...              3   
                       Fundamentos del Pamec : bases para la calidad e...              2   
                       Las convulsiones                                                2   
                       Las bases del dibujo.                                           2   
                       La verdad sea dicha serie.                                      2   
            18-29      Código civil : comentado                                       13   
                       Biología                                                       10   
                       Dere

Villavicencio


Cuenta_titulo  \
Tratamiento Rango_edad TITULO                                                              
sr.         40-49      Formación integral y competencias : pensamiento...              5   
                       Adolescente violentos : con los otros, con ello...              4   
                       Adolescentes en riesgo : casos prácticos y estr...              4   
                       Intervención con familias y atención a menores ...              4   
                       La causa de los niños                                           4   
                       Los miedos de los niños : cómo descubrirlos a t...              3   
                       Elogio de la dificultad y otros ensayos                         3   
                       El guión story : sustancia, estructura, estilo ...              3   
                       La memoria del uno y la memoria del otro : inco...              3   
                       Los ciclos de aprendizaje : un camino para comb...              3   

                                                                           Cuenta_grupo  
Tratamiento Rango_edad TITULO                                                            
sr.         40-49      Formación integral y competencias : pensamiento...           108  
                       Adolescente violentos : con los otros, con ello...           108  
                       Adolescentes en riesgo : casos prácticos y estr...           108  
                       Intervención con familias y atención a menores ...           108  
                       La causa de los niños                                        108  
                       Los miedos de los niños : cómo descubrirlos a t...           108  
                       Elogio de la dificultad y otros ensayos                      108  
                       El guión story : sustancia, estructura, estilo ...           108  
                       La memoria del uno y la memoria del otro : inco...           108  
                       Los ciclos de aprendizaje : un camino para comb...           108